# Shor's Algorithm

For the number 15, anyway…

In [51]:
# Imports of the Qiskit basic functionalities
%matplotlib inline
from qiskit import QuantumCircuit, transpile
from qiskit.providers.basic_provider import BasicSimulator
from qiskit.visualization import plot_histogram

In [52]:
def qft_dagger(n):
    qc = QuantumCircuit(n)
    for qubit in range(n//2):
        qc.swap(qubit, n-qubit-1)
    for j in range(n):
        for m in range(j):
            qc.cp(-np.pi/float(2**(j-m)), m, j)
        qc.h(j)
    return qc

def c_amod15(a, power):
    U = QuantumCircuit(4)   
    for iteration in range(power):
        U = U.compose(amod15(a))
    U = U.to_gate()
    U.name = "%i^%i mod 15" % (a, power)
    c_U = U.control()
    return c_U

def amod15(a):
    U = QuantumCircuit(4)
    if a in [2, 7, 8, 11, 13]:
        if a == 2:
            U.swap(0, 1)
            U.swap(1, 2)
            U.swap(2, 3)
        elif a == 7:
            U.swap(2, 3)
            U.swap(1, 2)
            U.swap(0, 1)
            U.x([1, 2])
        elif a == 8:
            U.swap(0, 3)
            U.swap(1, 2)
            U.x([1, 3])
        elif a == 11:
            U.swap(0, 3)
            U.swap(1, 2)
            U.x([0, 2])
        elif a == 13:
            U.swap(0, 1)
            U.swap(1, 2)
            U.swap(2, 3)
            U.x([0, 1])
    return U

def qpe_amod15(a):
    n_count = 8
    qc = QuantumCircuit(n_count + 4, n_count)
    for q in range(n_count):
        qc.h(q)
    qc.x(3+n_count)
    for q in range(n_count):
        qc.append(c_amod15(a, 2**q), [q] + [i+n_count for i in range(4)])
    qc.append(qft_dagger(n_count), range(n_count))
    qc.measure(range(n_count), range(n_count))
    return qc

def simulate_shors(N):
    for a in range(2, N):
        if gcd(a, N) == 1:
            qc = qpe_amod15(a)
            
            qc.draw(output='mpl')
            
            # Specify the backend (the simulator)
            simulator = BasicSimulator()

            # Transpile the circuit for the simulator
            compiled_circuit = transpile(qc, simulator)

            # Run the compiled circuit
            job = simulator.run(compiled_circuit)

            # Get the result
            result = job.result()

            counts = result.get_counts()
            # Print the counts
            print(counts)

            # Plot the histogram
            plot_histogram(counts)
            
            # Post-processing
            measured_phases = []
            for output in counts:
                decimal = int(output, 2)
                phase = decimal / (2**8)
                measured_phases.append(phase)

            fractions = [Fraction(phase).limit_denominator(N) for phase in measured_phases]
            rs = [frac.denominator for frac in fractions]

            # Remove duplicates and sort
            rs = list(set(rs))
            rs.sort()

            # Attempt to find the factors
            for r in rs:
                if r % 2 == 0:
                    guess1 = gcd(a**(r//2) - 1, N)
                    guess2 = gcd(a**(r//2) + 1, N)
                    if guess1 != 1 and guess1 != N:
                        print(f"Factors of {N} found: {guess1} and {N // guess1}")
                        return
                    if guess2 != 1 and guess2 != N:
                        print(f"Factors of {N} found: {guess2} and {N // guess2}")
                        return
            
            break

def main():
    N = 15
    print(f"Factors of {N}:")
    simulate_shors(N)

if __name__ == "__main__":
    main()


Factors of 15:
{'01000000': 265, '00000000': 246, '10000000': 253, '11000000': 260}
Factors of 15 found: 3 and 5
